In [1]:
import os

from openai import AsyncOpenAI

from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.pydantic_ai import OpenInferenceSpanProcessor
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

from pydantic import BaseModel, Field
from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.models.openai import OpenAIChatModel

PHOENIX_HTTP_ENDPOINT="http://localhost:9041/v1/traces"
PHOENIX_GRPC_ENDPOINT="http://localhost:4317"

tracer_provider = TracerProvider()
trace.set_tracer_provider(tracer_provider)

# Add the OpenInference span processor
# If you are using a local instance without auth, ignore these headers
headers = {"Authorization": f"Bearer YOUR_CUSTOM_KEY"}
exporter = OTLPSpanExporter(endpoint=PHOENIX_HTTP_ENDPOINT, headers=headers)

tracer_provider.add_span_processor(OpenInferenceSpanProcessor())
tracer_provider.add_span_processor(SimpleSpanProcessor(exporter))


In [2]:
# Setup Model
MODEL="Qwen3-VL-30B-A3B-Instruct"

# Passing AsyncOpenAI currently causes following error
# 'ASyncHttpxClientWrapper' object has no attribute '_state'
# llm_client=AsyncOpenAI(
#     base_url="http://localhost:9001/v1",
#     api_key="",
# )

provider = OpenAIProvider(
    # openai_client=llm_client,
    base_url="http://localhost:9001/v1",
    api_key="",
)
model = OpenAIChatModel(MODEL, provider=provider)

In [3]:
# Simple Example
class WeatherInfo(BaseModel):
    location: str
    temperature: float = Field(description="Temperature in Celsius")
    condition: str
    humidity: int = Field(description="Humidity percentage")

# Create an agent with system prompts and tools
weather_agent = Agent(
    model=model,
    output_type=WeatherInfo,
    system_prompt="You are a helpful weather assistant. Always provide accurate weather information.",
    instrument=True
)

@weather_agent.tool
async def get_weather_data(ctx: RunContext[None], location: str) -> str:
    """Get current weather data for a location."""
    # This is a placeholder - use a real weather API
    mock_data = {
        "temperature": 22.5,
        "condition": "partly cloudy",
        "humidity": 65
    }
    return f"Weather in {location}: {mock_data}"

# Run the agent with tool usage

In [4]:
result = await weather_agent.run("What's the weather like in Paris?")
print(result)

AgentRunResult(output=WeatherInfo(location='Paris', temperature=22.5, condition='partly cloudy', humidity=65))
